In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random as rd
import array
from math import sqrt

In [2]:
dataTable = pd.read_csv('cardio_train.csv', delimiter=';').drop('id', 1)

In [3]:
dataTable.head()

,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
0,18393,2,168,62.0,110,80,1,1,0,0,1,0
1,20228,1,156,85.0,140,90,3,1,0,0,1,1
2,18857,1,165,64.0,130,70,3,1,0,0,0,1
3,17623,2,169,82.0,150,100,1,1,0,0,1,1
4,17474,1,156,56.0,100,60,1,1,0,0,0,0


### Cleaning Dataset

In [4]:
def ageInYears(dataTable):
    ageInYears = []
    for i in range(len(dataTable)):
        ageInYears.append(int(round(dataTable.iloc[i]['age']/365)))
    return ageInYears

In [5]:
ageInYears = ageInYears(dataTable)
dataTable['age_years'] = ageInYears

In [6]:
cols = dataTable.columns.tolist()
cols.insert(0, cols.pop(cols.index('age_years')))
cols.insert(0, cols.pop(cols.index('age')))
dataTable = dataTable.reindex(columns= cols)
dataTable.columns = ['Age', 'Age_years', 'Gender', 'Height', 'Weight', 'Systolic_bp', 'Diastolic_bp', 'Cholesterol', 'Glucose', 'Smoking', 'Alcohol_intake', 'Physical_activity', 'Cardio_disease' ]

In [7]:
dataTable.head()

,Age,Age_years,Gender,Height,Weight,Systolic_bp,Diastolic_bp,Cholesterol,Glucose,Smoking,Alcohol_intake,Physical_activity,Cardio_disease
0,18393,50,2,168,62.0,110,80,1,1,0,0,1,0
1,20228,55,1,156,85.0,140,90,3,1,0,0,1,1
2,18857,52,1,165,64.0,130,70,3,1,0,0,0,1
3,17623,48,2,169,82.0,150,100,1,1,0,0,1,1
4,17474,48,1,156,56.0,100,60,1,1,0,0,0,0
